### MONITORING RULES

In [297]:
!pip install awswrangler

### EXCEL FILES WITH MAPEs OF TOP 15 PAYERS ARE UPLOADED

In [298]:
import awswrangler as wr
import boto3
import pandas as pd
from io import BytesIO
import numpy as np
from datetime import datetime, timedelta
from io import BytesIO # Para el WA
from io import StringIO
import joblib
import os

In [299]:
# S3 client
s3 = boto3.client('s3')

In [300]:
#Bucket and folder prefix
bucket_name = 'viamericas-datalake-dev-us-east-1-283731589572-analytics'
prefix_ABTv3_update = 'ABTv3_update/top_payers_abt_v3update/'

In [301]:
# I get list of folders in the bucket
def list_folders(bucket_name, prefix):
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix, Delimiter='/')
    folders = [prefix['Prefix'].split('/')[-2] for prefix in response.get('CommonPrefixes', [])]
    return folders

In [302]:
# I call a function to list folders
folders_v3 = list_folders(bucket_name, prefix_ABTv3_update)

In [303]:
# I read the xlsx and consolidate

def read_files(bucket_name, prefix):
    dfs = []
    
    # Iterar sobre cada carpeta
    for folder_name in folders_v3:
        # Obtener la lista de objetos en la carpeta
        objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=f'{prefix}{folder_name}/')['Contents']
        
        # Buscar archivos xlsx en la carpeta
        excel_objects = [obj for obj in objects if obj['Key'].endswith('.xlsx') and '7d' not in obj['Key']]
        
        if excel_objects:
            print(excel_objects)
            # Leer el primer archivo xlsx encontrado
            obj = s3.get_object(Bucket=bucket_name, Key=excel_objects[0]['Key'])
            excel_data = obj['Body'].read() # No se puede leer con pandas directo de S3
            
            df = pd.read_excel(BytesIO(excel_data))
            # Agrego columna con el nombre de la carpeta
            df['folder_name'] = folder_name
            # Agregar el df a la lista
            dfs.append(df)
        else:
            print(f"No se encontraron archivos xlsx en la carpeta {folder_name}.")
    
    if dfs:
        # Concateno todo
        consolidated_df = pd.concat(dfs, ignore_index=True)
        return consolidated_df
    else:
        print("No se encontraron archivos xlsx en ninguna de las carpetas especificadas.")
        return None

In [304]:
# I call function to read xlsx and consolidate into one DF
df_top15 = read_files(bucket_name, prefix_ABTv3_update)

[{'Key': 'ABTv3_update/top_payers_abt_v3update/BANCO AGRICOLA_EL SALVADOR/predicciones_test_2d_BANCO AGRICOLA_EL SALVADOR.xlsx', 'LastModified': datetime.datetime(2024, 3, 25, 18, 45, 53, tzinfo=tzlocal()), 'ETag': '"9b406073e87517287809516d839f8a2d"', 'Size': 18494, 'StorageClass': 'STANDARD'}]
[{'Key': 'ABTv3_update/top_payers_abt_v3update/BANCO ATLANTIDA_HONDURAS/predicciones_test_2d_BANCO ATLANTIDA_HONDURAS.xlsx', 'LastModified': datetime.datetime(2024, 3, 25, 17, 14, 50, tzinfo=tzlocal()), 'ETag': '"efab2b35f556266e631250662095466d"', 'Size': 18913, 'StorageClass': 'STANDARD'}]
[{'Key': 'ABTv3_update/top_payers_abt_v3update/BANCO DE OCCIDENTE_HONDURAS/predicciones_test_2d_BANCO DE OCCIDENTE_HONDURAS.xlsx', 'LastModified': datetime.datetime(2024, 3, 25, 19, 18, 47, tzinfo=tzlocal()), 'ETag': '"6246eb42efc092e74e315d514e726e4c"', 'Size': 18856, 'StorageClass': 'STANDARD'}]
[{'Key': 'ABTv3_update/top_payers_abt_v3update/BANCO DE ORO (BDO)_PHILIPPINES/predicciones_test_2d_BANCO DE ORO

In [305]:
df_top15.folder_name.nunique()

15

In [306]:
df_top15.date.max()

Timestamp('2024-02-03 00:00:00')

In [307]:
##DATE OF ANALYSIS
##HASTA 1 DIA ANTES DE LA PREDICCION QUE YA TIENE MAPE##
df_top15=df_top15.loc[df_top15.date<'2024-01-30']

In [308]:
df_top15.head()

,date,valor_real,valor_predicho,mape,error_abs,folder_name
0,2023-06-22,333877.5,383497.896918,14.861857,49620.396918,BANCO AGRICOLA_EL SALVADOR
1,2023-06-23,472255.0,526814.668818,11.553010,54559.668818,BANCO AGRICOLA_EL SALVADOR
2,2023-06-24,471284.0,489445.019909,3.853519,18161.019909,BANCO AGRICOLA_EL SALVADOR
3,2023-06-25,387389.0,409110.539878,5.607165,21721.539878,BANCO AGRICOLA_EL SALVADOR
4,2023-06-26,398244.0,409621.536758,2.856926,11377.536758,BANCO AGRICOLA_EL SALVADOR


In [309]:
df_top15.folder_name.unique()

array(['BANCO AGRICOLA_EL SALVADOR', 'BANCO ATLANTIDA_HONDURAS',
       'BANCO DE OCCIDENTE_HONDURAS', 'BANCO DE ORO (BDO)_PHILIPPINES',
       'BANCO INDUSTRIAL_GUATEMALA', 'BANCO RENDIMENTO_BRAZIL',
       'BANCOPPEL (APPRIZA)_MEXICO', 'BANORTE (UT)_MEXICO',
       'BANRURAL (RYT)_GUATEMALA', 'BBVA - BANCOMER (BTS)_MEXICO',
       'CARIBE EXPRESS_DOMINICAN REPUBLIC', 'ELEKTRA (MEXICO)_MEXICO',
       'ENVIOS CON CLABE (APPRIZA)_MEXICO', 'G Y T CONTINENTAL_GUATEMALA',
       'TELECOMM TELEGRAFOS (APPRIZA)_MEXICO'], dtype=object)

In [310]:
df_top15['date'] = pd.to_datetime(df_top15['date'])

In [311]:
##WE REPLACE THE NEGATIVE PREDICTED VALUES
valor_a_reemplazar = 0
df_top15['valor_predicho'] = df_top15['valor_predicho'].apply(lambda x: valor_a_reemplazar if x < 0 else x)

In [312]:
df_top15 = df_top15.sort_values(by=['folder_name', 'date'])

In [313]:
##WE OBTAIN THE MOVING AVERAGE OF THE MAPE OF THE PREVIOUS 7 DAYS.
df_top15['mape_7_days_avg'] = df_top15.groupby('folder_name')['mape'].rolling(window=7, min_periods=5).mean().reset_index(level=0, drop=True).shift(1)

In [314]:
df_top15

,date,valor_real,valor_predicho,mape,error_abs,folder_name,mape_7_days_avg
0,2023-06-22,333877.50,383497.896918,14.861857,49620.396918,BANCO AGRICOLA_EL SALVADOR,NaN
1,2023-06-23,472255.00,526814.668818,11.553010,54559.668818,BANCO AGRICOLA_EL SALVADOR,NaN
2,2023-06-24,471284.00,489445.019909,3.853519,18161.019909,BANCO AGRICOLA_EL SALVADOR,NaN
3,2023-06-25,387389.00,409110.539878,5.607165,21721.539878,BANCO AGRICOLA_EL SALVADOR,NaN
4,2023-06-26,398244.00,409621.536758,2.856926,11377.536758,BANCO AGRICOLA_EL SALVADOR,NaN
...,...,...,...,...,...,...,...
3395,2024-01-25,402488.65,421819.511674,4.802834,19330.861674,TELECOMM TELEGRAFOS (APPRIZA)_MEXICO,8.985139
3396,2024-01-26,439061.39,441922.341655,0.651606,2860.951655,TELECOMM TELEGRAFOS (APPRIZA)_MEXICO,8.055590
3397,2024-01-27,423683.10,462898.462382,9.255824,39215.362382,TELECOMM TELEGRAFOS (APPRIZA)_MEXICO,6.996590
3398,2024-01-28,527858.74,495858.589271,6.062256,32000.150729,TELECOMM TELEGRAFOS (APPRIZA)_MEXICO,7.607823


### DEFINITION OF RULES

### CONDITION 1: THAT THE 7-DAY MOVING AVERAGE OF THE LAST TWO MONDAYS GROWS:

In [315]:
##MAPE AVERAGE IN TEST
##REFERENCE HISTORICAL MAPEs ARE CALCULATED BETWEEN 01/01/2021 AND 12/18/23
mape_mean_period = df_top15[(df_top15['date'] >= '2023-06-22') & (df_top15['date'] <= '2023-12-18')].groupby('folder_name')['mape'].mean()

# FUNCTION TO CALCULATE THE MOVING AVERAGE OF THE LAST 7 DAYS EXCLUDING VALUES 10 TIMES OR MORE HIGHER TAHN THE AVERAGE MAPE.
def mape_7_days_avg(group):
    
    high_mape_values = group[group['mape'] >= 10 * mape_mean_period[group.name]]
    group.loc[high_mape_values.index, 'mape'] = None  # Replace high values with None

    #Calculate the moving average of the last 7 days for the reference period (06/2023 - 12-2023)
    group['mape_7_days_avg'] = group.groupby('folder_name')['mape'].rolling(window=7, min_periods=5).mean().reset_index(level=0, drop=True).shift(1)

    return group

# Apply the mape_7_days_avg function to each group

data_processed = df_top15.groupby('folder_name').apply(mape_7_days_avg)

# Reset index
data_processed.reset_index(drop=True, inplace=True)

#Apply the grouping function and obtain the variable CONDITION 1 for each 'folder_name'.

def check_growth(group):
    ###DETECT THE LAST MONDAY IN THE SERIES
    last_monday = group[group['date'].dt.dayofweek == 0].max()['date']
    ### EXTRAMING THE MAPEs MOBILE AVERAGE FOR THE 7 DAYS PRIOR TO THAT MONDAY
    val_mape_1_week = group[group['date'] == last_monday]['mape_7_days_avg'].iloc[0]
    ###DETECT THE PREVIOUS MONDAY
    previous_monday = last_monday - pd.DateOffset(7)
    ###OBTAINING THE MOVING AVERAGE OF MAPS FOR THE PREVIOUS SEVEN DAYS
    val_mape_2_week = group[group['date'] == previous_monday]['mape_7_days_avg'].iloc[0]
    ###DETECT THE MONDAY BEFORE THE SECOND MONDAY
    previous_previous_monday = previous_monday - pd.DateOffset(7)
     ###OBTAINING THE MOVING AVERAGE OF MAPES FOR THE PREVIOUS SEVEN DAYS
    val_mape_3_week = group[group['date'] == previous_previous_monday]['mape_7_days_avg'].iloc[0]

    # CREATE A DATAFRAME WITH ADDITIONAL COLUMNS
    result = pd.DataFrame({
        'date_last_check': [last_monday],
        'folder_name': [group['folder_name'].iloc[0]], # El nombre del folder_name es el mismo para todos los registros del grupo
        'val_mape_1_week': [val_mape_1_week],
        'val_mape_2_week': [val_mape_2_week],
        'val_mape_3_week': [val_mape_3_week]
    })

    # ADD CONDITION 1 COLUMN
    if val_mape_1_week > val_mape_2_week and val_mape_2_week > val_mape_3_week:
        result['flag_increasing_moving_averages'] = True
    else:
        result['flag_increasing_moving_averages'] = False

    return result

# APPLY THE CHECK_GROWTH FUNCTION AND CONCATENATE RESULTS
result = pd.concat([check_growth(group) for _, group in data_processed.groupby('folder_name')], ignore_index=True)

print(result)

   date_last_check                           folder_name  val_mape_1_week  \
0       2024-01-29            BANCO AGRICOLA_EL SALVADOR         5.705565   
1       2024-01-29              BANCO ATLANTIDA_HONDURAS         3.558499   
2       2024-01-29           BANCO DE OCCIDENTE_HONDURAS         8.739783   
3       2024-01-29        BANCO DE ORO (BDO)_PHILIPPINES        17.575626   
4       2024-01-29            BANCO INDUSTRIAL_GUATEMALA         5.023035   
5       2024-01-29               BANCO RENDIMENTO_BRAZIL        30.494050   
6       2024-01-29            BANCOPPEL (APPRIZA)_MEXICO         5.695510   
7       2024-01-29                   BANORTE (UT)_MEXICO         6.146157   
8       2024-01-29              BANRURAL (RYT)_GUATEMALA        10.464288   
9       2024-01-29          BBVA - BANCOMER (BTS)_MEXICO         5.794886   
10      2024-01-29     CARIBE EXPRESS_DOMINICAN REPUBLIC         7.365481   
11      2024-01-29               ELEKTRA (MEXICO)_MEXICO         7.109602   

/tmp/ipykernel_110/3081722859.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_processed = df_top15.groupby('folder_name').apply(mape_7_days_avg)


In [316]:
mape_mean_period2=data_processed[(data_processed['date'] >= '2023-06-22') & (data_processed['date'] <= '2023-12-18')].groupby('folder_name')['mape'].mean()
##AVERAGE MAPE OF THE TEST PERIOD (2023-06 2023-12) WITHOUT MAPEs > 10X TEST PERIOD
mape_mean_period2

folder_name
BANCO AGRICOLA_EL SALVADOR               7.094948
BANCO ATLANTIDA_HONDURAS                 8.089944
BANCO DE OCCIDENTE_HONDURAS              7.828954
BANCO DE ORO (BDO)_PHILIPPINES          15.246002
BANCO INDUSTRIAL_GUATEMALA               7.838171
BANCO RENDIMENTO_BRAZIL                 18.856583
BANCOPPEL (APPRIZA)_MEXICO               8.023823
BANORTE (UT)_MEXICO                      9.350056
BANRURAL (RYT)_GUATEMALA                 8.651675
BBVA - BANCOMER (BTS)_MEXICO             9.952609
CARIBE EXPRESS_DOMINICAN REPUBLIC        8.666772
ELEKTRA (MEXICO)_MEXICO                  6.994206
ENVIOS CON CLABE (APPRIZA)_MEXICO        8.032867
G Y T CONTINENTAL_GUATEMALA              9.788683
TELECOMM TELEGRAFOS (APPRIZA)_MEXICO    11.625459
Name: mape, dtype: float64

### CONDITION2: THE MAPE OF THE LAST WEEK IS NOT GREATER THAN THE MAXIMUM 7-DAY MOVING AVERAGE HISTORICAL MAXIMUM OF THE TEST PERIOD

In [317]:
mape_max_period=data_processed[(data_processed['date'] >= '2023-06-22') & (data_processed['date'] <= '2023-12-18')].groupby('folder_name')['mape_7_days_avg'].max()
mape_max_period

folder_name
BANCO AGRICOLA_EL SALVADOR              13.488821
BANCO ATLANTIDA_HONDURAS                16.010195
BANCO DE OCCIDENTE_HONDURAS             17.638946
BANCO DE ORO (BDO)_PHILIPPINES          28.174718
BANCO INDUSTRIAL_GUATEMALA              24.976621
BANCO RENDIMENTO_BRAZIL                 42.138393
BANCOPPEL (APPRIZA)_MEXICO              21.307336
BANORTE (UT)_MEXICO                     17.853906
BANRURAL (RYT)_GUATEMALA                18.573531
BBVA - BANCOMER (BTS)_MEXICO            18.435182
CARIBE EXPRESS_DOMINICAN REPUBLIC       20.647582
ELEKTRA (MEXICO)_MEXICO                 18.153615
ENVIOS CON CLABE (APPRIZA)_MEXICO       19.877006
G Y T CONTINENTAL_GUATEMALA             23.019552
TELECOMM TELEGRAFOS (APPRIZA)_MEXICO    24.395719
Name: mape_7_days_avg, dtype: float64

In [318]:
###WE ADD THE MAXIMUM OF THE 7-DAY MOVING AVERAGE OF THE MAPEs IN THE TEST PERIOD
df_monitoring=pd.merge(result, mape_max_period, on='folder_name', how='outer')

In [319]:
df_monitoring

,date_last_check,folder_name,val_mape_1_week,val_mape_2_week,val_mape_3_week,flag_increasing_moving_averages,mape_7_days_avg
0,2024-01-29,BANCO AGRICOLA_EL SALVADOR,5.705565,11.808818,9.431614,False,13.488821
1,2024-01-29,BANCO ATLANTIDA_HONDURAS,3.558499,5.389756,10.746015,False,16.010195
2,2024-01-29,BANCO DE OCCIDENTE_HONDURAS,8.739783,11.656978,13.328649,False,17.638946
3,2024-01-29,BANCO DE ORO (BDO)_PHILIPPINES,17.575626,14.833366,16.661164,False,28.174718
4,2024-01-29,BANCO INDUSTRIAL_GUATEMALA,5.023035,7.610178,9.756218,False,24.976621
5,2024-01-29,BANCO RENDIMENTO_BRAZIL,30.494050,25.152328,19.417284,True,42.138393
6,2024-01-29,BANCOPPEL (APPRIZA)_MEXICO,5.695510,8.668374,13.626734,False,21.307336
7,2024-01-29,BANORTE (UT)_MEXICO,6.146157,10.718542,10.743145,False,17.853906
8,2024-01-29,BANRURAL (RYT)_GUATEMALA,10.464288,14.021984,8.786395,False,18.573531
9,2024-01-29,BBVA - BANCOMER (BTS)_MEXICO,5.794886,11.143071,16.143505,False,18.435182


In [320]:
df_monitoring.rename(columns={'mape_7_days_avg': 'val_mape_max_mobile_7_days'}, inplace=True)

In [321]:
###GENERATE CONDITION 2 AND CONDITION 3
def generate_condition2(row):
    if  row['val_mape_1_week']  > row['val_mape_max_mobile_7_days'] * 1.05:
        return True
    else:
        return False

def generate_condition3(row):
    if  row['val_mape_1_week']  >  row['val_mape_max_mobile_7_days'] * 1.10:
        return True
    else:
        return False

# Aplicar la función a lo largo de las filas del DataFrame
df_monitoring['flag_activate_calibration'] = df_monitoring.apply(generate_condition2, axis=1)
df_monitoring['flag_activate_retraining'] = df_monitoring.apply(generate_condition3, axis=1)

In [322]:
df_monitoring

,date_last_check,folder_name,val_mape_1_week,val_mape_2_week,val_mape_3_week,flag_increasing_moving_averages,val_mape_max_mobile_7_days,flag_activate_calibration,flag_activate_retraining
0,2024-01-29,BANCO AGRICOLA_EL SALVADOR,5.705565,11.808818,9.431614,False,13.488821,False,False
1,2024-01-29,BANCO ATLANTIDA_HONDURAS,3.558499,5.389756,10.746015,False,16.010195,False,False
2,2024-01-29,BANCO DE OCCIDENTE_HONDURAS,8.739783,11.656978,13.328649,False,17.638946,False,False
3,2024-01-29,BANCO DE ORO (BDO)_PHILIPPINES,17.575626,14.833366,16.661164,False,28.174718,False,False
4,2024-01-29,BANCO INDUSTRIAL_GUATEMALA,5.023035,7.610178,9.756218,False,24.976621,False,False
5,2024-01-29,BANCO RENDIMENTO_BRAZIL,30.494050,25.152328,19.417284,True,42.138393,False,False
6,2024-01-29,BANCOPPEL (APPRIZA)_MEXICO,5.695510,8.668374,13.626734,False,21.307336,False,False
7,2024-01-29,BANORTE (UT)_MEXICO,6.146157,10.718542,10.743145,False,17.853906,False,False
8,2024-01-29,BANRURAL (RYT)_GUATEMALA,10.464288,14.021984,8.786395,False,18.573531,False,False
9,2024-01-29,BBVA - BANCOMER (BTS)_MEXICO,5.794886,11.143071,16.143505,False,18.435182,False,False


### RECOMMENDED ACTIONS

In [323]:
###RECOMMENDED ACTIONS ARE GENERATED

def actions_recommended(row):
    if  row['flag_increasing_moving_averages'] == True and row['flag_activate_calibration']==True and row['flag_activate_retraining']==False:
        return 'requires calibration'
    elif row['flag_increasing_moving_averages'] == True and row['flag_activate_retraining']==True:
        return 'requires re-training'
    else:
        return 'keep current model'

df_monitoring['desc_recommended_actions'] = df_monitoring.apply(actions_recommended, axis=1)

In [324]:
df_monitoring

,date_last_check,folder_name,val_mape_1_week,val_mape_2_week,val_mape_3_week,flag_increasing_moving_averages,val_mape_max_mobile_7_days,flag_activate_calibration,flag_activate_retraining,desc_recommended_actions
0,2024-01-29,BANCO AGRICOLA_EL SALVADOR,5.705565,11.808818,9.431614,False,13.488821,False,False,keep current model
1,2024-01-29,BANCO ATLANTIDA_HONDURAS,3.558499,5.389756,10.746015,False,16.010195,False,False,keep current model
2,2024-01-29,BANCO DE OCCIDENTE_HONDURAS,8.739783,11.656978,13.328649,False,17.638946,False,False,keep current model
3,2024-01-29,BANCO DE ORO (BDO)_PHILIPPINES,17.575626,14.833366,16.661164,False,28.174718,False,False,keep current model
4,2024-01-29,BANCO INDUSTRIAL_GUATEMALA,5.023035,7.610178,9.756218,False,24.976621,False,False,keep current model
5,2024-01-29,BANCO RENDIMENTO_BRAZIL,30.494050,25.152328,19.417284,True,42.138393,False,False,keep current model
6,2024-01-29,BANCOPPEL (APPRIZA)_MEXICO,5.695510,8.668374,13.626734,False,21.307336,False,False,keep current model
7,2024-01-29,BANORTE (UT)_MEXICO,6.146157,10.718542,10.743145,False,17.853906,False,False,keep current model
8,2024-01-29,BANRURAL (RYT)_GUATEMALA,10.464288,14.021984,8.786395,False,18.573531,False,False,keep current model
9,2024-01-29,BBVA - BANCOMER (BTS)_MEXICO,5.794886,11.143071,16.143505,False,18.435182,False,False,keep current model


### ADD ID COUNTRY, ID PAYER AND DESC PAYER, COUNTRY

In [325]:
###ojo que estamos sacando id_main_branch y id_country de este csv y no de la tabla viva##
### preguntar a Lauta, lo mismo hicimos con lo de envios con clabe##

In [326]:
##CARGAMOS LOS CSV DE LAS TABLAS UTILIZADAS EN PROD HASTA EL 09/05/2024 ENVIADAS POR 
## RAFAEL ROSAS
# Set up S3 client
client = boto3.client('s3')
bucket_name = 'viamericas-datalake-dev-us-east-1-283731589572-analytics'
path = 'Data_PROD_Abt/' # Folder under analysis

In [327]:
# Specify the CSV file key
csv_key = None

# List objects in the S3 path
response = client.list_objects(Bucket=bucket_name, Prefix=path)

# Find the CSV file in the S3 path
for obj in response.get('Contents', []):
    if obj['Key'].endswith('daily_check_gp.csv'):
        csv_key = obj['Key']
        break
        
# Check if CSV file is found
if csv_key is not None:
    # Read CSV content from S3
    csv_response = client.get_object(Bucket=bucket_name, Key=csv_key)
    csv_content = csv_response['Body'].read().decode('utf-8')

    # Transform CSV content to DataFrame
    data1 = pd.read_csv(StringIO(csv_content))
    data1['date'] = pd.to_datetime(data1['date']).dt.date
    print("CSV file loaded")
else:
    print("No CSV file found in the specified S3 path.")

CSV file loaded


In [328]:
data1.date.max()

datetime.date(2024, 5, 29)

In [329]:
aux=data1.loc[:,['payer','country','id_main_branch', 'id_country']]
aux = aux.drop_duplicates(subset=['id_country', 'id_main_branch'], keep='last')
aux['payer_country'] = aux['payer'] +'_'+ aux['country']
aux=aux.loc[:,['payer_country','id_main_branch', 'id_country']]

In [330]:
df_monitoring = pd.merge(df_monitoring, aux, left_on=['folder_name'],right_on=['payer_country'], how='inner')
df_monitoring[['payer', 'country']] = df_monitoring['payer_country'].str.split('_', expand=True)

In [331]:
df_monitoring = df_monitoring.rename(columns={'payer': 'des_payer', 'country': 'des_country', 'id_main_branch': 'id_payer'})
cols_to_eliminate = ['payer_country', 'folder_name']
df_monitoring = df_monitoring.drop(columns=cols_to_eliminate)

In [332]:
df_monitoring

,date_last_check,val_mape_1_week,val_mape_2_week,val_mape_3_week,flag_increasing_moving_averages,val_mape_max_mobile_7_days,flag_activate_calibration,flag_activate_retraining,desc_recommended_actions,id_payer,id_country,des_payer,des_country
0,2024-01-29,5.705565,11.808818,9.431614,False,13.488821,False,False,keep current model,T053,ELS,BANCO AGRICOLA,EL SALVADOR
1,2024-01-29,3.558499,5.389756,10.746015,False,16.010195,False,False,keep current model,T109,HON,BANCO ATLANTIDA,HONDURAS
2,2024-01-29,8.739783,11.656978,13.328649,False,17.638946,False,False,keep current model,T068,HON,BANCO DE OCCIDENTE,HONDURAS
3,2024-01-29,17.575626,14.833366,16.661164,False,28.174718,False,False,keep current model,T209,PHL,BANCO DE ORO (BDO),PHILIPPINES
4,2024-01-29,5.023035,7.610178,9.756218,False,24.976621,False,False,keep current model,T085,GUA,BANCO INDUSTRIAL,GUATEMALA
5,2024-01-29,30.494050,25.152328,19.417284,True,42.138393,False,False,keep current model,T056,BRA,BANCO RENDIMENTO,BRAZIL
6,2024-01-29,5.695510,8.668374,13.626734,False,21.307336,False,False,keep current model,T239,MEX,BANCOPPEL (APPRIZA),MEXICO
7,2024-01-29,6.146157,10.718542,10.743145,False,17.853906,False,False,keep current model,T160,MEX,BANORTE (UT),MEXICO
8,2024-01-29,10.464288,14.021984,8.786395,False,18.573531,False,False,keep current model,T228,GUA,BANRURAL (RYT),GUATEMALA
9,2024-01-29,5.794886,11.143071,16.143505,False,18.435182,False,False,keep current model,T057,MEX,BBVA - BANCOMER (BTS),MEXICO


In [333]:
# Nombre del archivo con la fecha
tbl_forecast_monitoring_mape=df_monitoring.copy()

tbl_forecast_monitoring_mape['date_last_check'] = tbl_forecast_monitoring_mape['date_last_check'].dt.strftime('%Y%m%d')
file_name = f"tbl_forecast_monitoring_mape_{tbl_forecast_monitoring_mape['date_last_check'].iloc[0]}.csv"

# Dejo el archivo en S3
bucket = 'viamericas-datalake-dev-us-east-1-283731589572-analytics'
prefix_abt = 'MONITORING_RULES'

# Ruta S3
s3_path = f"s3://{bucket}/{prefix_abt}/{file_name}"

# Guardar el DataFrame en CSV en S3
wr.s3.to_csv(tbl_forecast_monitoring_mape, path=s3_path, index=False)

{'paths': ['s3://viamericas-datalake-dev-us-east-1-283731589572-analytics/MONITORING_RULES/tbl_forecast_monitoring_mape_20240129.csv'],
 'partitions_values': {}}